In [2]:
import gzip
import ast
import json
import pandas as pd
from datetime import datetime
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [3]:
def crearDFJSON(ruta):
    data = []
    with gzip.open(ruta, 'rb') as f:
        for line in f:
            data.append(json.loads(line))
    f.close
    return pd.DataFrame(data)

def crearDFAST(ruta):
    data = []
    with gzip.open(ruta, 'rb') as f:
        for line in f:
            data.append(ast.literal_eval(line.decode('utf-8')))
    f.close
    return pd.DataFrame(data)

def desanidarDF(df, columna):
    df_desanidado = df.explode(columna)
    return df_desanidado

def desanidar(df, columna):
    dfDesanidado = pd.json_normalize(df[columna])
    df = df.reset_index(drop=True)
    df = pd.concat([df, dfDesanidado], axis=1)
    df = df.drop(columna, axis=1)
    return df

In [4]:
steamGamesDF = crearDFJSON('../data/steam_games.json.gz')
userReviews = crearDFAST('../data/user_reviews.json.gz')

In [5]:
steamGamesDF = steamGamesDF[['id', 'app_name', 'release_date']].dropna().drop_duplicates().reset_index(drop=True)

In [6]:
steamGamesDF.columns = ['item_id', 'name', 'date']

In [7]:
steamGamesDF['year'] = pd.to_datetime(steamGamesDF['date'], errors='coerce').dt.year
    
    # Eliminar filas con años no válidos
steamGamesDF = steamGamesDF.dropna(subset=['year'])

In [8]:
steamGamesDF = steamGamesDF.drop(['date'], axis=True)

In [9]:
userReviews = desanidarDF(userReviews, 'reviews')
userReviews = desanidar(userReviews, 'reviews')

In [10]:
columnas_eliminar = ['user_url', 'funny', 'posted', 'last_edited', 'helpful']
userReviews = userReviews.drop(columnas_eliminar, axis=1).reset_index(drop=True)

In [11]:
userReviews = userReviews.dropna()

In [12]:
userReviews = userReviews.drop_duplicates().reset_index(drop=True)

In [13]:

# Descargar los recursos necesarios para NLTK
nltk.download('vader_lexicon')

# Inicializar el analizador de sentimientos
sid = SentimentIntensityAnalyzer()

# Función para asignar el sentimiento a cada reseña
def get_sentiment(review):
    if pd.isnull(review):
        return 1  # Valor neutral si no hay reseña
    else:
        scores = sid.polarity_scores(review)
        compound_score = scores['compound']
        if compound_score >= 0.05:
            return 2  # Positivo
        elif compound_score <= -0.05:
            return 0  # Negativo
        else:
            return 1  # Neutral

# Aplicar la función a la columna 'review' y crear la columna 'sentiment_analysis'
userReviews['sentiment_analysis'] = userReviews['review'].apply(get_sentiment)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/konas/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [14]:
userReviews = userReviews.drop(['review'], axis=True)

In [15]:
userReviews

,user_id,item_id,recommend,sentiment_analysis
0,76561197970982479,1250,True,2
1,76561197970982479,22200,True,2
2,76561197970982479,43110,True,2
3,js41637,251610,True,2
4,js41637,227300,True,2
...,...,...,...,...
58426,76561198312638244,70,True,2
58427,76561198312638244,362890,True,2
58428,LydiaMorley,273110,True,2
58429,LydiaMorley,730,True,2


In [16]:
userReviews.columns = ['user_id', 'item_id', 'recommend', 'sentiment_analysis']

In [17]:
merged_df = pd.merge(userReviews, steamGamesDF, left_on='item_id', right_on='item_id', how='left')

In [18]:
merged_df = merged_df.dropna(subset=['year']).reset_index(drop=True)

In [19]:
columnas_eliminar = ['user_id','item_id']
merged_df = merged_df.drop(columnas_eliminar, axis=1).reset_index(drop=True)

In [20]:
merged_df

,recommend,sentiment_analysis,name,year
0,True,2,Killing Floor,2009.0
1,True,2,Zeno Clash,2009.0
2,True,2,Euro Truck Simulator 2,2013.0
3,True,2,"Papers, Please",2013.0
4,True,2,Risk of Rain,2013.0
...,...,...,...,...
49614,True,2,Half-Life,1998.0
49615,True,2,Black Mesa,2015.0
49616,True,2,Counter-Strike Nexon: Zombies,2014.0
49617,True,2,Counter-Strike: Global Offensive,2012.0


In [21]:
merged_df.to_parquet('UsersRecommendParquet.parquet', index=False)

In [22]:
def UsersRecommend(año, df):
    # Filtrar los datos según el año dado
    datos_año = df[df['year'].astype(int) == año]
    
    # Seleccionar solo las filas donde recommend sea True y sentiment_analysis sea 2 o 1
    recomendados = datos_año[(datos_año['recommend'] == True) & (datos_año['sentiment_analysis'].isin([1, 2]))]
    
    # Contar el número de recomendaciones para cada juego
    conteo_recomendaciones = recomendados['name'].value_counts()

    # Convertir el objeto zip en una lista y luego ordenar los juegos según el número de recomendaciones en orden descendente
    juegos_ordenados = list(conteo_recomendaciones.items())
    
    # Seleccionar los tres primeros juegos
    top_3 = [{"Puesto {}".format(i+1): juego[0]} for i, juego in enumerate(juegos_ordenados[:3])]
    
    return top_3

In [23]:
year = 2008  # Año dado
top_games = UsersRecommend(year, merged_df)
print(top_games)

[{'Puesto 1': 'Call of Duty: World at War'}, {'Puesto 2': 'AudioSurf'}, {'Puesto 3': 'SPORE™'}]


In [24]:
def UsersNotRecommend(año, df):
    # Filtrar los datos según el año dado
    datos_año = df[df['year'].astype(int) == año]
    
    # Seleccionar solo las filas donde recommend sea True y sentiment_analysis sea 2 o 1
    recomendados = datos_año[(datos_año['recommend'] == False) & (datos_año['sentiment_analysis'].isin([0]))]

    # Contar el número de recomendaciones para cada juego
    conteo_recomendaciones = recomendados['name'].value_counts()
    
    # Convertir el objeto zip en una lista y luego ordenar los juegos según el número de recomendaciones en orden descendente
    juegos_ordenados = list(conteo_recomendaciones.items())
    
    # Seleccionar los tres primeros juegos
    top_3 = [{"Puesto {}".format(i+1): juego[0]} for i, juego in enumerate(juegos_ordenados[:3])]
    
    return top_3

In [25]:
year = 2008  # Año dado
bottom_games = UsersNotRecommend(year, merged_df)
print(bottom_games)

[{'Puesto 1': "Far Cry® 2: Fortune's Edition"}, {'Puesto 2': 'SPORE™'}, {'Puesto 3': 'X3: Terran Conflict'}]


In [26]:
def sentiment_analysis(year: int):
    # Filtrar el DataFrame por el año proporcionado
    df_filtered = merged_df[merged_df['year'] == year]
    print(df_filtered.count())
    # Contar la cantidad de registros por cada categoría de análisis de sentimiento
    sentiment_counts = df_filtered['sentiment_analysis'].value_counts()
    
    # Mapear los valores numéricos de la categoría de análisis de sentimiento a sus respectivos nombres
    sentiment_mapping = {2: 'Positive', 1: 'Neutral', 0: 'Negative'}
    
    # Crear el diccionario de retorno con los valores mapeados
    return {sentiment_mapping[key]: value for key, value in sentiment_counts.items()}

    

In [27]:
year = 2010  # Año dado
print(sentiment_analysis(year))

recommend             1978
sentiment_analysis    1978
name                  1978
year                  1978
dtype: int64
{'Positive': 1311, 'Neutral': 357, 'Negative': 310}
